In [8]:
import pandas as pd
import re
import os
# from openpyxl import load_workbook
# file_path = re.sub(r'^AC-?', '', product_code)

In [9]:
# Specify the directory
directory = "data/formula/"

# List all files and directories in the specified directory
files_and_dirs = os.listdir(directory)

# If you only want files, not directories, you can filter the list
files = [f for f in files_and_dirs if os.path.isfile(os.path.join(directory, f))]

In [10]:
df_ref = pd.read_excel('data/01-refrence.xlsx')
selected_columns = df_ref[['کد قدیم کالا', 'نام کالا']]

# Converting the selected columns to a list of dictionaries
list_of_dicts = selected_columns.to_dict(orient='records')

# Extract codes from file names
codes = [file.split('.')[0] for file in files]  # Removes the '.xls' part

# Create a dictionary to map old-codes to نام کالا
code_to_name_map = pd.Series(df_ref['نام کالا'].values, index=df_ref['کد قدیم کالا']).to_dict()

# Find corresponding نام کالا for each code
names = [code_to_name_map.get(code, 'Not Found') for code in codes]

In [11]:
def convert_product(df_output, df_temp, product_code, product_name):
    df_formula = pd.read_excel(f'data/formula/{product_code}.xls')
    # Assuming df_formula is your DataFrame
    total_sum = df_formula['درصد'].sum()

    if total_sum != 100:
        # Normalize the values to sum up to 100
        df_formula['درصد'] = (df_formula['درصد'] / total_sum) * 100

        # Round to 2 decimal places
        df_formula['درصد'] = df_formula['درصد'].round(2)

        # Adjust the last value to ensure the total sum is exactly 100
        last_index = df_formula.columns.get_loc('درصد')
        df_formula.iloc[-1, last_index] = 100 - df_formula.iloc[:-1, last_index].sum()

        # Round the last element again to handle floating-point precision issues
        df_formula.iloc[-1, last_index] = round(df_formula.iloc[-1, last_index], 2)

    df_temp['کالای مصرفی'] = df_formula['کد ماده اولیه']
    df_temp['ضریب مصرف استاندارد'] = df_formula['درصد']
    df_temp['کد کالای محصول'] = product_code
    df_temp['عنوان BOM'] = product_name
    # Assuming df_output and df_temp are your DataFrames
    # Remove columns that are completely empty or all NA from both DataFrames
    df_output = df_output.dropna(axis=1, how='all')
    df_temp = df_temp.dropna(axis=1, how='all')

    # Now perform the concatenation
    df_output = pd.concat([df_output, df_temp], ignore_index=True)
    return df_output

In [12]:
df_output = pd.DataFrame(columns=['کد کالای محصول', 'کالای مصرفی', 'ضریب مصرف استاندارد', 'عنوان BOM'])

for i in range(len(names)):
    df_temp = pd.DataFrame(columns=['کد کالای محصول', 'کالای مصرفی', 'ضریب مصرف استاندارد', 'عنوان BOM'])
    product_code = codes[i]
    product_name = names[i]
    df_output = convert_product(df_output, df_temp, product_code, product_name)


In [13]:
df_alter = pd.read_excel('data/list alternative 1402-10-06 رنگ مایع.xlsx')
# List of columns from df_alter to be added to df_output
columns_to_add = ['alter_code_1', 'alter_code_2', 'alter_code_3', 'alter_code_4','alter_code_5']

# # Perform the merge
# df_output = pd.merge(df_output, df_alter[['main_code'] + columns_to_add], left_on='کالای مصرفی', right_on='main_code', how='left')

# # Optionally, you can drop the 'main_code' column if it's not needed
# df_output.drop(columns=['main_code'], inplace=True)

for item in df_output['کالای مصرفی']:
    if item in df_alter['کالای مصرفی'].values:
        matching_row = df_alter[df_alter['کالای مصرفی'] == item][selected_columns]
        df_output = pd.merge(df_output, matching_row, left_on='کالای مصرفی', right_index=True, how='left')


KeyError: 'کالای مصرفی'

In [ ]:
df_output.columns

Index(['کد کالای محصول', 'کالای مصرفی', 'ضریب مصرف استاندارد', 'عنوان BOM',
       'alter_code_1', 'alter_code_2', 'alter_code_3', 'alter_code_4',
       'alter_code_5'],
      dtype='object')

In [ ]:
df_change = pd.read_excel('data/code-change.xlsx')
# Create a mapping dictionary from df_change
change_dict = pd.Series(df_change['کد کالا'].values, index=df_change['مشخصه فنی']).to_dict()

# List of columns in df_output to be updated
columns_to_update = ['کالای مصرفی', 'alter_code_1', 'alter_code_2', 'alter_code_3', 'alter_code_4', 'alter_code_5']

# Update the values in df_output based on the mapping
for col in columns_to_update:
    df_output[col] = df_output[col].map(change_dict).fillna(df_output[col])

# Create a mapping dictionary from df_change
change_dict = pd.Series(df_change['کد کالا'].values, index=df_change['مشخصه فنی']).to_dict()

# Transform the 'کد کالای محصول' column and save it to a new column
df_output['کد کالای محصول جدید'] = df_output['کد کالای محصول'].map(change_dict).fillna(df_output['کد کالای محصول'])

df_output.to_excel('output.xlsx', index=False)